# Zonal Stats

By Cascade Tuholske, June 2020

Notebook finds zonal stats of populations for give geographies. <br><br>
**NOTE** CRS should be epsg:4326 for everything!

In [1]:
#### Dependencies
import numpy as np
import pandas as pd
import rasterio
import geopandas as gpd
from rasterstats import zonal_stats, gen_zonal_stats
from glob import glob

In [2]:
#### File Paths & FNs
DATA_PATH = '/Users/cascade/Github/PopGridCompare/data/'

In [3]:
#### Run on Nigeria GDAM 
polys_fn = DATA_PATH+'raw/GDAM/gadm36_ESP_shp/gadm36_ESP_4.shp'
polys = gpd.read_file(polys_fn)
col = 'GID_4' # gdam level
fn_out = DATA_PATH+'interim/ESP4_stats.shp' #updatea

In [4]:
# subset, be sure to check the admin level
polys = polys[['geometry', col]]

In [5]:
# Git tif files
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*.tif')
rst_fns

['/Users/cascade/Github/PopGridCompare/data/interim/GHS15_matched.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/LS15_matched.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/ESRI16_matched.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/WP16_matched.tif']

In [6]:
def zone_loop(polys_in, rst_list, stats_type):
    """ Function loops through rasters, calcs zonal_stats and returns stats as a data frame.
    Args:
        polys_in = polygons
        rst_list = list of paths & fns of rasters
        stats_type = stats type for each poly gone (see zonal stats)
    """
    
    # copy polys to write out
    polys_out = polys_in.copy()
    
    for rst in rst_list:
        
        # Get data name
        data = rst.split(DATA_PATH+'interim/')[1].split('_matched.tif')[0]
        print('Started', data)
        
        # Run zonal stats
        zs_feats = zonal_stats(polys_in, rst, stats=stats_type, geojson_out=True)
        zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=polys_in.crs)
        
        # Rename columns and merge
        zgdf = zgdf.rename(columns={stats_type: data+'_'+stats_type})
        
        polys_out = polys_out.merge(zgdf[[col, data+'_'+stats_type]], on = col, how = 'inner')
    
    return polys_out


In [7]:
# Run zonal stats loop
polys_sum = zone_loop(polys, rst_fns, 'sum')
polys_sum.head()

Started GHS15
Started LS15
Started ESRI16
Started WP16


,geometry,GID_4,GHS15_sum,LS15_sum,ESRI16_sum,WP16_sum
0,"POLYGON ((-2.19652605056757 37.27780532836914,...",ESP.1.1.1.1_1,787.941356,1311.0,820.0,831.010681
1,POLYGON ((-2.183551073074341 37.38243103027344...,ESP.1.1.1.2_1,10733.399701,8406.0,10597.0,12236.562500
2,POLYGON ((-2.619179964065495 37.24225234985363...,ESP.1.1.1.3_1,630.648293,4040.0,325.0,664.089478
3,POLYGON ((-2.030813932418823 37.43074035644537...,ESP.1.1.1.4_1,4677.951219,5241.0,6662.0,5065.504395
4,POLYGON ((-2.396323919296265 37.34402847290039...,ESP.1.1.1.5_1,362.962382,249.0,850.0,494.843597


In [8]:
# variances
polys_sum['var'] = polys_sum.iloc[:,2:6].var(axis = 1)

In [9]:
# std (population level)
polys_sum['std'] = polys_sum.iloc[:,2:6].std(axis = 1, ddof=0)

In [10]:
# range
polys_sum['range'] = polys_sum.iloc[:,2:6].max(axis = 1) - polys_sum.iloc[:,2:6].min(axis = 1)

In [11]:
polys_sum.head()

,geometry,GID_4,GHS15_sum,LS15_sum,ESRI16_sum,WP16_sum,var,std,range
0,"POLYGON ((-2.19652605056757 37.27780532836914,...",ESP.1.1.1.1_1,787.941356,1311.0,820.0,831.010681,6.233875e+04,216.226885,523.058644
1,POLYGON ((-2.183551073074341 37.38243103027344...,ESP.1.1.1.2_1,10733.399701,8406.0,10597.0,12236.562500,2.488062e+06,1366.033215,3830.562500
2,POLYGON ((-2.619179964065495 37.24225234985363...,ESP.1.1.1.3_1,630.648293,4040.0,325.0,664.089478,3.085933e+06,1521.331587,3715.000000
3,POLYGON ((-2.030813932418823 37.43074035644537...,ESP.1.1.1.4_1,4677.951219,5241.0,6662.0,5065.504395,7.502091e+05,750.104530,1984.048781
4,POLYGON ((-2.396323919296265 37.34402847290039...,ESP.1.1.1.5_1,362.962382,249.0,850.0,494.843597,6.794682e+04,225.743475,601.000000


In [12]:
#### Save it out
gpd_out = gpd.GeoDataFrame(polys_sum)
gpd_out.to_file(fn_out)

# Old Code

In [ ]:
#### Functions
def zonal_func(polys_in, rst_in, stats_type, save, fn_out = None):
    """ Runs zonal stats on a set of polygons for a given raster, see rasterstats for stats type.
    Returns geodata frame
    Args:
        polys = polygons as a shape file read into memory
        rst_fn = path to raster file to run zonal stats on
        stats_type = stats type for each poly gone (see zonal stats)
        save = True will save out a fail
        fn_out = file name and path to save out shape files
    """
    
    # Run Zonal Stats & Set to gpd df
    zs_feats = zonal_stats(polys_in, rst_in, stats=stats_type, geojson_out=True)
    zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=polys_in.crs)
    
    if save == True:
        zgdf.to_file(fn_out) 
    
    return zgdf

In [ ]:
#### Run Zonal Stats
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*.tif')
fn_out = DATA_PATH+'interim/NGA_2_LS15.shp'
rst_fn = rst_fns[0]
zstats = zonal_func(polys, rst_fn, 'sum', save = True, fn_out =fn_out)